In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split
import numpy as np
#physical_devices = tf.config.list_physical_devices('GPU') 
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
# Download the data, if not already on disk and load it as numpy arrays
lfw_people = fetch_lfw_people(min_faces_per_person=70, resize=0.4)

In [3]:
# introspect the images arrays to find the shapes (for plotting)
n_samples, h, w = lfw_people.images.shape
# for machine learning we use the 2 data directly (as relative pixel
# positions info is ignored by this model)
X = lfw_people.data
n_features = X.shape[1]
# the label to predict is the id of the person
y = lfw_people.target
target_names = lfw_people.target_names
n_classes = target_names.shape[0]
y = tf.one_hot(y, depth=n_classes) ####
print("Total dataset size:")
print("n_samples: %d" % n_samples)
print("n_features: %d" % n_features)
print("n_classes: %d" % n_classes)

Total dataset size:
n_samples: 1288
n_features: 1850
n_classes: 7


In [4]:
#normalise data
X = X/255
X

array([[0.99607843, 0.99607843, 0.9869281 , ..., 0.34248367, 0.3477124 ,
        0.33986926],
       [0.15555556, 0.19738561, 0.18431373, ..., 0.4614379 , 0.4509804 ,
        0.52418303],
       [0.3503268 , 0.40784314, 0.49411765, ..., 0.68758166, 0.7189542 ,
        0.7176471 ],
       ...,
       [0.3372549 , 0.3150327 , 0.29281044, ..., 0.17254902, 0.19477125,
        0.1751634 ],
       [0.19738561, 0.25751632, 0.34509805, ..., 0.77254903, 0.70326793,
        0.65228754],
       [0.11764706, 0.10588235, 0.12810458, ..., 0.13725491, 0.13856208,
        0.23921569]], dtype=float32)

In [5]:
# Split into a training set and a test set using a stratified k fold
X_train, X_test, y_train, y_test = train_test_split(X, y.numpy(), test_size=0.25, random_state=42)

In [6]:
from tensorflow import keras

In [7]:
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1850,)))
model.add(tf.keras.layers.Dense(units = 512, activation='relu'))
model.add(tf.keras.layers.Dense(units = 256, activation='relu'))
model.add(tf.keras.layers.Dense(units = 192, activation='relu'))
model.add(tf.keras.layers.Dense(units = 128, activation='relu'))
model.add(tf.keras.layers.Dense(units = 7, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               947712    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 192)               49344     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               24704     
_________________________________________________________________
dense_4 (Dense)              (None, 7)                 903       
Total params: 1,153,991
Trainable params: 1,153,991
Non-trainable params: 0
_________________________________________________________________


In [8]:
model.compile(optimizer='adam', loss = tf.losses.CategoricalCrossentropy(), metrics=['accuracy'])

In [9]:
history = model.fit(X_train, y_train,epochs=50, validation_split = 0.2)

Epoch 1/50
25/25 [==============================] - 1s 10ms/step - loss: 1.8399 - accuracy: 0.3484 - val_loss: 1.6496 - val_accuracy: 0.4124
Epoch 2/50
25/25 [==============================] - 0s 4ms/step - loss: 1.7297 - accuracy: 0.3938 - val_loss: 1.6513 - val_accuracy: 0.4124
Epoch 3/50
25/25 [==============================] - 0s 4ms/step - loss: 1.6843 - accuracy: 0.3951 - val_loss: 1.6020 - val_accuracy: 0.4381
Epoch 4/50
25/25 [==============================] - 0s 4ms/step - loss: 1.6160 - accuracy: 0.4119 - val_loss: 1.5971 - val_accuracy: 0.4588
Epoch 5/50
25/25 [==============================] - 0s 4ms/step - loss: 1.5500 - accuracy: 0.4391 - val_loss: 1.5237 - val_accuracy: 0.4897
Epoch 6/50
25/25 [==============================] - 0s 4ms/step - loss: 1.5287 - accuracy: 0.4534 - val_loss: 1.4384 - val_accuracy: 0.5000
Epoch 7/50
25/25 [==============================] - 0s 4ms/step - loss: 1.3203 - accuracy: 0.4922 - val_loss: 1.5601 - val_accuracy: 0.4536
Epoch 8/50
25/25 [=

In [10]:
#Evaluate the model on the test data
loss, acc = model.evaluate(X_test, y_test)

11/11 [==============================] - 0s 3ms/step - loss: 0.6767 - accuracy: 0.7919
